This notebook will be used to analyse the different neighbourhoods in Mumbai and choose the best cluster to start a new restaurant. 

In [166]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import folium
import requests
from IPython.core.display import HTML
from pandas import json_normalize
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
print('LIBRARIES IMPORTED')

LIBRARIES IMPORTED


In [167]:
df=pd.read_excel('/Users/dylan.kakkanad/Downloads/clustering.xlsx',sheet_name='Mumbai')
df.columns=['Postalcode','Neighbourhood','Latitude','Longitude']
display(df)
print(df.shape)

,Postalcode,Neighbourhood,Latitude,Longitude
0,400001,Bazargate,18.9297,72.8336
1,400001,M.P.T.,18.9957,72.8589
2,400001,Mumbai G.P.O.,18.9388,72.8370
3,400001,Stock Exchange,18.9300,72.8334
4,400001,Tajmahal,18.9217,72.8330
...,...,...,...,...
223,400102,Oshiwara,19.1483,72.8405
224,400103,Mandapeshwar,19.2475,72.8552
225,400104,Bangur Nagar,19.1655,72.8323
226,400104,Goregaon,19.1663,72.8526


(228, 4)


In [282]:
df.dropna(axis=0,inplace=True)
df
print('Ther are ', df.shape[0], 'neighbourhoods in Mumbai')

Ther are  228 neighbourhoods in Mumbai


Lets define a function thatll give us co-ordinates of the address we need

In [169]:
def location(address):
    geolocator=Nominatim(user_agent='toronto_agent')
    location=geolocator.geocode(address)
    latitude=location.latitude
    longitude=location.longitude
    print('The co-ordinates(lat,long) of',address,'are:',latitude,longitude)


In [170]:
location('Mumbai, India')

The co-ordinates(lat,long) of Mumbai, India are: 18.9387711 72.8353355


In [171]:
Mumbai_Map=folium.Map(location=(18.9387711,72.8353355),zoom_start=10)
HTML(Mumbai_Map._repr_html_())
for lat,long,neighbourhood in zip(df['Latitude'],df['Longitude'],df['Neighbourhood']):
    label='{}'.format(neighbourhood)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(Mumbai_Map)
Mumbai_Map


In [172]:
print(location('Mumbai,maharashtra'))

The co-ordinates(lat,long) of Mumbai,maharashtra are: 18.9387711 72.8353355
None


<h3>Lets now use the foursquare API to explore the venues located in Mumbai<h3/>

In [173]:
CLIENT_ID = 'VY0FKC4VLFL5OLZ0EXUB1UBSIKMA0XBRTPEMP05XWQOGNE2A' # your Foursquare ID
CLIENT_SECRET = 'I5OTTLPCDAJGBAAYNCW4LVQVH5FWB5301KAV4D5N0CMHNQJU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VY0FKC4VLFL5OLZ0EXUB1UBSIKMA0XBRTPEMP05XWQOGNE2A
CLIENT_SECRET:I5OTTLPCDAJGBAAYNCW4LVQVH5FWB5301KAV4D5N0CMHNQJU


In [209]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [210]:
Mumbai=getNearbyVenues(names=df['Neighbourhood'],latitudes=df['Latitude'], longitudes=df['Longitude'])

Bazargate 
M.P.T. 
Mumbai G.P.O.
Stock Exchange 
Tajmahal 
Town Hall 
Kalbadevi 
Ramwadi 
S. C. Court 
Thakurdwar 
B.P.Lane 
Mandvi 
Masjid 
Null Bazar 
Ambewadi 
Charni Road 
Chaupati 
Girgaon 
Madhavbaug 
Opera House 
Asvini 
Colaba 
Holiday Camp 
V.W.T.C. 
Malabar Hill 
Bharat Nagar 
Grant Road 
S V Marg 
Tardeo 
Falkland Road 
J.J.Hospital 
Kamathipura 
M A Marg 
Mumbai Central 
Chinchbunder 
Noor Baug 
Princess Dock 
Dockyard Road 
Mazgaon Dock 
Mazgaon 
V K Bhavan 
Agripada 
Chinchpokli 
Haines Road 
Jacob Circle 
BEST STaff Quarters 
Chamarbaug 
Haffkin Institute 
Lal Baug 
Parel Naka 
Parel 
Delisle Road 
Dadar Colony 
Dadar 
Naigaon 
Sewri 
Kapad Bazar 
Mahim Bazar 
Mahim East 
Mahim 
Mori Road 
Dharavi Road 
Dharavi 
Worli Naka 
Worli 
Matunga 
Central Building 
Churchgate 
Marine Lines 
Nariman Point 
New Yogakshema 
Chunabhatti 
Raoli Camp 
Sion 
Transit Camp 
Nehru Nagar 
New Prabhadevi Road 
Prabhadevi 
Cumballa Hill 
Cumballa Sea Face 
Dr Deshmukh Marg 
Gowalia Tank 
V J

In [227]:
Mumbai

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bazargate,18.9297,72.8336,Trishna,18.928619,72.832356,Seafood Restaurant
1,Bazargate,18.9297,72.8336,La Folie Patisserie,18.928504,72.832315,Dessert Shop
2,Bazargate,18.9297,72.8336,Filter,18.928312,72.832643,Gift Shop
3,Bazargate,18.9297,72.8336,The Nutcracker,18.928509,72.833683,Breakfast Spot
4,Bazargate,18.9297,72.8336,Golden Dragon,18.928479,72.832493,Chinese Restaurant
...,...,...,...,...,...,...,...
2810,Motilal Nagar,19.1575,72.8383,Talwalkars,19.160206,72.839556,Gym
2811,Motilal Nagar,19.1575,72.8383,Carnival Cinemas,19.157458,72.835196,Multiplex
2812,Motilal Nagar,19.1575,72.8383,Ozone Activity,19.159483,72.841448,Athletics & Sports
2813,Motilal Nagar,19.1575,72.8383,Goregaon BEST Bus Depot,19.154601,72.835448,Bus Station


In [228]:
Mumbai.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
A I Staff Colony,1,1,1,1,1,1
Aarey Colony,3,3,3,3,3,3
Agripada,5,5,5,5,5,5
Ambewadi,26,26,26,26,26,26
Andheri East,11,11,11,11,11,11
...,...,...,...,...,...,...
Worli Colony,12,12,12,12,12,12
Worli Naka,12,12,12,12,12,12
Worli Police Camp,8,8,8,8,8,8


In [229]:
# one hot encoding
Mumbai_onehot = pd.get_dummies(Mumbai[['Venue Category']], prefix="", prefix_sep="")
Mumbai_onehot
#Mumbai_onehot.drop(['Neighbourhood'],axis=1,inplace=True) 
Mumbai_onehot.insert(loc=0, column='Neighbourhood', value=Mumbai['Neighbourhood'] )
Mumbai_onehot.shape
Mumbai_onehot

,Neighbourhood,ATM,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,...,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Bazargate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bazargate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bazargate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bazargate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bazargate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2810,Motilal Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2811,Motilal Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2812,Motilal Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2813,Motilal Nagar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [230]:
Mumbai_grouped=Mumbai_onehot.groupby('Neighbourhood').mean().reset_index()
Mumbai_grouped

,Neighbourhood,ATM,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,...,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,A I Staff Colony,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Aarey Colony,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,Agripada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,Ambewadi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038462,0.000,...,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.038462,0.0
4,Andheri East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Worli Colony,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
213,Worli Naka,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
214,Worli Police Camp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.125,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
215,Worli Sea Face,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [231]:
Mumbai_grouped.shape

(217, 202)

In [232]:
num_top_venues = 5

for hood in Mumbai_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Mumbai_grouped[Mumbai_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A I Staff Colony ----
                     venue  freq
0            Grocery Store   1.0
1                      ATM   0.0
2  New American Restaurant   0.0
3      Monument / Landmark   0.0
4                    Motel   0.0


----Aarey Colony ----
                     venue  freq
0                    Hotel  0.33
1              Film Studio  0.33
2     Gym / Fitness Center  0.33
3                      ATM  0.00
4  New American Restaurant  0.00


----Agripada ----
                venue  freq
0            Platform   0.2
1         Coffee Shop   0.2
2                Bank   0.2
3              Bakery   0.2
4  Athletics & Sports   0.2


----Ambewadi ----
               venue  freq
0          Juice Bar  0.12
1  Indian Restaurant  0.08
2     Ice Cream Shop  0.08
3               Café  0.08
4      Train Station  0.08


----Andheri East ----
                  venue  freq
0                 Hotel  0.27
1    Seafood Restaurant  0.18
2  Fast Food Restaurant  0.18
3     Indian Restaurant  0.09
4         

In [233]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [239]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Mumbai_grouped['Neighbourhood']

for ind in np.arange(Mumbai_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Mumbai_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A I Staff Colony,Grocery Store,Zoo,Dim Sum Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
1,Aarey Colony,Hotel,Film Studio,Gym / Fitness Center,Diner,Fish Market,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
2,Agripada,Athletics & Sports,Platform,Coffee Shop,Bank,Bakery,Donut Shop,Fish Market,Film Studio,Field,Fast Food Restaurant
3,Ambewadi,Juice Bar,Restaurant,Café,Ice Cream Shop,Train Station,Indian Restaurant,Coffee Shop,Pizza Place,Food Truck,Multicuisine Indian Restaurant
4,Andheri East,Hotel,Fast Food Restaurant,Seafood Restaurant,Smoke Shop,Restaurant,Indian Restaurant,Gym,Field,Farmers Market,Falafel Restaurant


In [240]:
kclusters = 7

Mumbai_grouped_clustering = Mumbai_grouped.drop('Neighbourhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:39] 

array([4, 5, 6, 6, 3, 3, 3, 6, 3, 6, 6, 6, 1, 1, 6, 6, 1, 1, 6, 6, 6, 5,
       1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], dtype=int32)

In [241]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Mumbai_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Mumbai_merged = Mumbai_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
Mumbai_merged.dropna(inplace=True)
Mumbai_merged['Cluster Labels'].astype(int)

Mumbai_merged.head() # check the last columns!

,Postalcode,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,400001,Bazargate,18.9297,72.8336,6.0,Indian Restaurant,Seafood Restaurant,Café,Dessert Shop,Coffee Shop,Chinese Restaurant,Restaurant,Juice Bar,Boutique,Breakfast Spot
1,400001,M.P.T.,18.9957,72.8589,6.0,Harbor / Marina,Zoo,Dim Sum Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
2,400001,Mumbai G.P.O.,18.9388,72.8370,1.0,Indian Restaurant,Café,Irani Cafe,Chinese Restaurant,Lounge,Multiplex,Seafood Restaurant,Coffee Shop,Bar,Train Station
3,400001,Stock Exchange,18.9300,72.8334,6.0,Indian Restaurant,Café,Seafood Restaurant,Dessert Shop,Coffee Shop,Chinese Restaurant,Restaurant,Juice Bar,Boutique,Breakfast Spot
4,400001,Tajmahal,18.9217,72.8330,6.0,Café,Hotel,Indian Restaurant,Coffee Shop,Bar,Cocktail Bar,Pub,Pizza Place,Falafel Restaurant,Spa


In [242]:
map_clusters = folium.Map(location=[18.9387711, 72.8353355], zoom_start=11)
HTML(map_clusters._repr_html_())
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Mumbai_merged['Latitude'], Mumbai_merged['Longitude'], Mumbai_merged['Neighbourhood'], Mumbai_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Let us now examine each cluster<h3/>

<h3>Cluster 0<h3/>

In [272]:
Cluster_0=Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 0, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]
print(Cluster_0.shape)
Cluster_0

(2, 11)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,Raoli Camp,Vegetarian / Vegan Restaurant,Train Station,Zoo,Dim Sum Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
73,Sion,Vegetarian / Vegan Restaurant,Train Station,Zoo,Dim Sum Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


<h3>Cluster 1<h3/>

In [287]:
Cluster_1=Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 1, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]
print(Cluster_1.shape)
Cluster_1

(64, 11)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Mumbai G.P.O.,Indian Restaurant,Café,Irani Cafe,Chinese Restaurant,Lounge,Multiplex,Seafood Restaurant,Coffee Shop,Bar,Train Station
6,Kalbadevi,Indian Restaurant,Food,Market,Cheese Shop,Jewelry Store,Snack Place,Zoo,Donut Shop,Film Studio,Field
7,Ramwadi,Indian Restaurant,Food,Market,Cheese Shop,Jewelry Store,Donut Shop,Fish Market,Film Studio,Field,Fast Food Restaurant
9,Thakurdwar,Indian Restaurant,Food,Market,Cheese Shop,Jewelry Store,Donut Shop,Fish Market,Film Studio,Field,Fast Food Restaurant
10,B.P.Lane,Indian Restaurant,Ice Cream Shop,Fast Food Restaurant,Restaurant,Electronics Store,Juice Bar,Italian Restaurant,Pizza Place,Beach,Steakhouse
...,...,...,...,...,...,...,...,...,...,...,...
201,Sahakar Bhavan,Indian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Restaurant,Convenience Store,Cricket Ground,Donut Shop,Film Studio,Field,Fast Food Restaurant
207,Tilak Nagar,Indian Restaurant,Food Truck,Bakery,Zoo,Donut Shop,Fish Market,Film Studio,Field,Fast Food Restaurant,Farmers Market
214,Seepz,Restaurant,Pizza Place,Airport Terminal,Indian Restaurant,Vegetarian / Vegan Restaurant,Bus Station,Zoo,Field,Fast Food Restaurant,Farmers Market
217,Vidyanagari,Indian Restaurant,Burger Joint,Fast Food Restaurant,Ice Cream Shop,Pet Store,Asian Restaurant,Pharmacy,Bus Station,Restaurant,Sandwich Place


<h3>Cluster 2<h3/>

In [274]:
Cluster_2=Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 2, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]
print(Cluster_2.shape)
Cluster_2

(11, 11)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Mazgaon Dock,ATM,Bus Stop,Bakery,Government Building,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
39,Mazgaon,ATM,Bus Stop,Bakery,Government Building,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
112,D.M. Colony,ATM,Cafeteria,Diner,Fish Market,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
113,Shivaji Nagar,ATM,Electronics Store,Diner,Fish Market,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
184,Psm Colony,ATM,Indian Restaurant,Bakery,Donut Shop,Fish Market,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
185,Vikhroli,ATM,Indian Restaurant,Bakery,Donut Shop,Fish Market,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
195,Kannamwar Nagar,ATM,Comedy Club,Asian Restaurant,Convenience Store,Donut Shop,Fish Market,Film Studio,Field,Fast Food Restaurant,Farmers Market
203,Govandi,ATM,Cafeteria,Indian Restaurant,Gym,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
204,T.F.Donar,ATM,Cafeteria,Indian Restaurant,Gym,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
215,Malad East,ATM,Persian Restaurant,Smoke Shop,Gym,Dim Sum Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


<h3>Cluster 3<h3/>

In [275]:
Cluster_3=Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 3, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]
print(Cluster_3.shape)
Cluster_3

(20, 11)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Town Hall,Fast Food Restaurant,Coffee Shop,Bar,Indian Restaurant,Asian Restaurant,Café,Middle Eastern Restaurant,Chinese Restaurant,Multiplex,Bus Station
31,Kamathipura,Department Store,Fast Food Restaurant,Indian Restaurant,Asian Restaurant,Dessert Shop,Arts & Crafts Store,Chinese Restaurant,Middle Eastern Restaurant,Dumpling Restaurant,Farmers Market
33,Mumbai Central,Department Store,Fast Food Restaurant,Snack Place,Ice Cream Shop,Restaurant,Vegetarian / Vegan Restaurant,Train Station,Pizza Place,Movie Theater,Farmers Market
42,Chinchpokli,Asian Restaurant,Fast Food Restaurant,Multiplex,Flower Shop,Fish Market,Film Studio,Field,Farmers Market,Falafel Restaurant,Event Space
56,Kapad Bazar,Fast Food Restaurant,Indian Restaurant,Breakfast Spot,Movie Theater,Hotel Bar,Pizza Place,Dessert Shop,Bank,Cheese Shop,Café
57,Mahim Bazar,Fast Food Restaurant,Indian Restaurant,Breakfast Spot,Movie Theater,Hotel Bar,Pizza Place,Dessert Shop,Bank,Cheese Shop,Café
58,Mahim East,Fast Food Restaurant,Indian Restaurant,Breakfast Spot,Movie Theater,Hotel Bar,Pizza Place,Dessert Shop,Bank,Cheese Shop,Café
59,Mahim,Fast Food Restaurant,Indian Restaurant,Breakfast Spot,Movie Theater,Hotel Bar,Pizza Place,Dessert Shop,Bank,Cheese Shop,Café
125,Andheri,Hotel Bar,Hotel,Indian Restaurant,Pizza Place,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Department Store,Burger Joint,Donut Shop
131,IRLA,Ice Cream Shop,Fast Food Restaurant,Seafood Restaurant,Miscellaneous Shop,Multiplex,Electronics Store,Film Studio,Field,Farmers Market,Falafel Restaurant


<h3>Cluster 4<h3/>

In [276]:
Cluster_4=Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 4, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]
print(Cluster_4.shape)
Cluster_4

(1, 11)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
89,A I Staff Colony,Grocery Store,Zoo,Dim Sum Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store


<h3>Cluster 5<h3/>

In [277]:
Cluster_5=Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 5, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]
print(Cluster_5.shape)
Cluster_5

(7, 11)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Malabar Hill,Park,Boarding House,Dessert Shop,Donut Shop,Fish Market,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
137,H.M.P. School,Asian Restaurant,Gym / Fitness Center,Snack Place,Cosmetics Shop,Donut Shop,Fish Market,Film Studio,Field,Fast Food Restaurant,Farmers Market
150,Aarey Colony,Hotel,Film Studio,Gym / Fitness Center,Diner,Fish Market,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
174,Best Staff Colony,Gym / Fitness Center,Park,Restaurant,Zoo,Dim Sum Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
175,Pant Nagar,Gym / Fitness Center,Park,Restaurant,Zoo,Dim Sum Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
176,R.A.Nagar,Gym / Fitness Center,Park,Restaurant,Zoo,Dim Sum Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
191,Mhada Colony,Playground,Gym / Fitness Center,Zoo,Dim Sum Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


<h3>Cluster 6<h3/>

In [288]:
Cluster_6=Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 6, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]
print(Cluster_6.shape)
Cluster_6.head(60)

(112, 11)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bazargate,Indian Restaurant,Seafood Restaurant,Café,Dessert Shop,Coffee Shop,Chinese Restaurant,Restaurant,Juice Bar,Boutique,Breakfast Spot
1,M.P.T.,Harbor / Marina,Zoo,Dim Sum Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
3,Stock Exchange,Indian Restaurant,Café,Seafood Restaurant,Dessert Shop,Coffee Shop,Chinese Restaurant,Restaurant,Juice Bar,Boutique,Breakfast Spot
4,Tajmahal,Café,Hotel,Indian Restaurant,Coffee Shop,Bar,Cocktail Bar,Pub,Pizza Place,Falafel Restaurant,Spa
8,S. C. Court,Gym,Indian Restaurant,Video Store,Food Truck,Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Hotel,Falafel Restaurant,Farmers Market
12,Masjid,Convenience Store,Harbor / Marina,Hotel,Indian Restaurant,Electronics Store,Restaurant,Middle Eastern Restaurant,Dumpling Restaurant,Fish Market,Film Studio
14,Ambewadi,Juice Bar,Restaurant,Café,Ice Cream Shop,Train Station,Indian Restaurant,Coffee Shop,Pizza Place,Food Truck,Multicuisine Indian Restaurant
15,Charni Road,Juice Bar,Restaurant,Café,Ice Cream Shop,Train Station,Indian Restaurant,Coffee Shop,Pizza Place,Food Truck,Multicuisine Indian Restaurant
16,Chaupati,Ice Cream Shop,Juice Bar,Restaurant,Café,Beach,Train Station,Indian Restaurant,Italian Restaurant,Opera House,Pizza Place
17,Girgaon,Indian Restaurant,Snack Place,Fast Food Restaurant,Juice Bar,Electronics Store,Breakfast Spot,Italian Restaurant,Indie Movie Theater,Café,Vegetarian / Vegan Restaurant
